In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
from datetime import datetime
import pandas as pd
import json
import threading 

In [2]:
base_url='https://www.vaidam.com/doctors/india?page=1'

In [3]:
html_text=requests.get(base_url).text

In [6]:
html_text

'<!DOCTYPE html> <html lang="en" dir="ltr"> <meta http-equiv="content-type" content="text/html;charset=UTF-8" /> <head> <meta charset="utf-8" />  <link rel="shortlink" href="https://www.vaidam.com/doctors/india/?page=1" /> <title>Best Doctors in India | Top 10 Surgeons in India</title>  <meta http-equiv="X-UA-Compatible" content="IE=edge"> <meta content="width=device-width, initial-scale=1" name="viewport" /> <meta name="description" content="List of top 10 doctors in india. Read doctor profile, Get FREE Medical opinion and Quote. Get assistance in Visa, Medical travel and Treatment from best surgeons through Vaidam Health."> <meta name="author" content="Vaidam Health">  <meta property="og:url" content="http://www.vaidam.com/doctors/india?page=1"> <meta property="og:type" content="website"> <meta property="og:title" content="Best Doctors in India | Top 10 Surgeons in India"> <meta property="og:description" content="List of top 10 doctors in india. Read doctor profile, Get FREE Medical 

In [4]:
soup=BeautifulSoup(html_text,'lxml')

In [5]:
list_of_countries=list(map(lambda x: x.find('a').get('href'),soup.find('ul',id='Search-By-Country').find_all('li')))
list_of_countries

['https://www.vaidam.com/doctors/india',
 'https://www.vaidam.com/doctors/thailand',
 'https://www.vaidam.com/doctors/turkey',
 'https://www.vaidam.com/doctors/united-arab-emirates',
 'https://www.vaidam.com/doctors/south-korea',
 'https://www.vaidam.com/doctors/malaysia',
 'https://www.vaidam.com/doctors/germany',
 'https://www.vaidam.com/doctors/singapore',
 'https://www.vaidam.com/doctors/south-africa',
 'https://www.vaidam.com/doctors/israel',
 'https://www.vaidam.com/doctors/spain',
 'https://www.vaidam.com/doctors/egypt',
 'https://www.vaidam.com/doctors/tunisia',
 'https://www.vaidam.com/doctors/cyprus']

In [7]:
last_page_number=int(soup.find('ul',class_='pagination').find('a',title='Last').get('href').replace('?page=',''))
last_page_number

851

In [22]:

doctors=soup.find('div',class_='item-list').find_all('h2',class_='primary-heading-sm')


In [26]:
doctor_urls=list(map(lambda x: x.find('a').get('href'),doctors))

In [27]:
doctor_urls

['https://www.vaidam.com/doctors/dr-y-k-mishra-cardiac-surgeon',
 'https://www.vaidam.com/doctors/dr-sandeep-vaishya-neurosurgery',
 'https://www.vaidam.com/doctors/dr-rajeev-verma-orthopedics',
 'https://www.vaidam.com/doctors/dr-ajay-kaul-cardiac-surgeon',
 'https://www.vaidam.com/doctors/dr-naresh-trehan-cardiology',
 'https://www.vaidam.com/doctors/dr-vinod-raina-medical-oncology',
 'https://www.vaidam.com/doctors/dr-arun-saroha-neurosurgery',
 'https://www.vaidam.com/doctors/dr-z-s-meharwal-cardiac-surgeon',
 'https://www.vaidam.com/doctors/dr-bhaba-nanda-das-cardio-thoracic-vascular-surgeon',
 'https://www.vaidam.com/doctors/dr-ramesh-sarin']

In [8]:
all_doctors=[]
lock = threading.Lock()
pages=[]

def process_country(country):
    html_text = requests.get(country).text
    soup = BeautifulSoup(html_text, 'lxml')
    #try:
    try:
        last_page_number = int(soup.find('ul',class_='pagination').find('a',title='Last').get('href').replace('?page=',''))
        print(last_page_number)
    except:
        last_page_number = 1
    for page in range(1, last_page_number+1):
        html_text = requests.get(country+'?page='+str(page)).text

        soup = BeautifulSoup(html_text, 'lxml')

        try:
            doctors = soup.find('div',class_='item-list').find_all('h2',class_='primary-heading-sm')
            doctor_urls = list(map(lambda x: x.find('a').get('href'), doctors))
            
            for doctor in doctor_urls:
                if doctor not in all_doctors:
                    lock.acquire()
                    all_doctors.append(doctor)
                    pages.append(page)
                    lock.release()
        except Exception as e:
            print(e)
            print(country)
            print(page)
            print('error')
        if page % 5 == 0:
            print(f"{page}th page done for {country} done!")
    # except Exception as e:
    #     print(e)
    #     try:
    #         doctors = soup.find('div',class_='item-list').find_all('h2',class_='primary-heading-sm')
    #         doctor_urls = list(map(lambda x: x.find('a').get('href'), doctors))
    #         all_doctors.extend(doctor_urls)
    #     except Exception as e:
    #         print(e)
    #         print(country)
    #         print('error')
    print(f"{country} done!")

list_of_threads = []
for country in list_of_countries:
    thread = threading.Thread(target=process_country, args=(country,))
    list_of_threads.append(thread)
    thread.start()

for thread in list_of_threads:
    thread.join()


https://www.vaidam.com/doctors/cyprus done!
5
16
56
52
74
9
108
114
94
53
851
https://www.vaidam.com/doctors/egypt done!
https://www.vaidam.com/doctors/tunisia done!
5th page done for https://www.vaidam.com/doctors/spain done!
https://www.vaidam.com/doctors/spain done!
5th page done for https://www.vaidam.com/doctors/malaysia done!
5th page done for https://www.vaidam.com/doctors/singapore done!
5th page done for https://www.vaidam.com/doctors/south-korea done!
5th page done for https://www.vaidam.com/doctors/thailand done!
5th page done for https://www.vaidam.com/doctors/israel done!
5th page done for https://www.vaidam.com/doctors/united-arab-emirates done!
5th page done for https://www.vaidam.com/doctors/south-africa done!
5th page done for https://www.vaidam.com/doctors/turkey done!
5th page done for https://www.vaidam.com/doctors/germany done!
5th page done for https://www.vaidam.com/doctors/india done!
https://www.vaidam.com/doctors/israel done!
10th page done for https://www.vai

In [7]:
len(all_doctors)

144

In [12]:
all_doctors[:20]

['https://www.vaidam.com/doctors/dr-ali-enver-kurt',
 'https://www.vaidam.com/doctors/dr-ali-kizilkanat',
 'https://www.vaidam.com/doctors/dr-hayat-izel',
 'https://www.vaidam.com/doctors/op-dr-murat-onal',
 'https://www.vaidam.com/doctors/dr-yong-quek-wei',
 'https://www.vaidam.com/doctors/dr-timothy-lee',
 'https://www.vaidam.com/doctors/dr-tan-yew-oo',
 'https://www.vaidam.com/doctors/assoc-prof-low-boon-yong',
 'https://www.vaidam.com/doctors/prof-tan-kok-chai',
 'https://www.vaidam.com/doctors/clin-prof-fock-kwong-ming',
 'https://www.vaidam.com/doctors/dr-sulaiman-bin-yusof',
 'https://www.vaidam.com/doctors/dr-ching-kwok-choy',
 'https://www.vaidam.com/doctors/dr-c-sivathasa',
 'https://www.vaidam.com/doctors/dr-chumpon-chantharakulpongsa',
 'https://www.vaidam.com/doctors/dr-mohamed-badie',
 'https://www.vaidam.com/doctors/dr-ahmed-shams',
 'https://www.vaidam.com/doctors/dr-hesham-abbas',
 'https://www.vaidam.com/doctors/dr-ahmed-al-amir',
 'https://www.vaidam.com/doctors/dr-a

In [13]:
data = {"URLs": all_doctors}
df = pd.DataFrame(data)

In [3]:
output_file = "urls.xlsx"
df.to_excel(output_file, index=False)

NameError: name 'df' is not defined

In [2]:
df=pd.read_excel('urls.xlsx')
all_doctors=df['URLs'].values

In [3]:
all_doctors=list(all_doctors)

In [4]:
results = []
#lock = threading.Lock()
from concurrent.futures import ThreadPoolExecutor
i=0

def scrape_doctor_info(url):
    global i
    try:
        
        #print(url)
        response = requests.get(url)
        html_text = response.text
        soup = BeautifulSoup(html_text, 'lxml')
        
        
        
        name = soup.find("h1", class_="doc-name").text.strip()
        location = soup.find("p", class_="doc-location").text.strip()
        specialty = soup.find("div", class_="doc-specialization").text.strip()
        experience = soup.find("p", class_="doc-exp text-primary").text.strip()
        description = soup.find("ul", class_="")
        description_text = description.text.strip() if description else ""
        image_url = "https://www.vaidam.com"+soup.find("img", class_="doc-pic").get("src")
        url = url

        # Store the scraped data in a dictionary
        doctor_info = {
            "name": name,
            "location": location,
            "specialty": specialty,
            "experience": experience,
            "description": description_text,
            "url": url,
            "image_url": image_url,
        }
        

        # Acquire the lock to safely append to the results list
        #lock.acquire()
        results.append(doctor_info)
        i+=1
        #lock.release()

        if i % 50 == 0:
            print(f"{i}th doctor done!")
        #print(f"{i}th doctor done!")

    except Exception as e:
        # Handle any errors that occur during the request
        print("Error scraping URL:", url)
        print(e)


max_threads = 10

# Create a thread pool executor
with ThreadPoolExecutor(max_workers=max_threads) as executor:
    # Submit the scraping tasks to the executor
    futures = [executor.submit(scrape_doctor_info, url) for url in all_doctors]

    # Wait for all tasks to complete
    for future in futures:
        future.result()

50th doctor done!
100th doctor done!
150th doctor done!
200th doctor done!
250th doctor done!
300th doctor done!
350th doctor done!
400th doctor done!
450th doctor done!
500th doctor done!
550th doctor done!
600th doctor done!
650th doctor done!
700th doctor done!
750th doctor done!
800th doctor done!
850th doctor done!
900th doctor done!
950th doctor done!
1000th doctor done!
1050th doctor done!
1100th doctor done!
1150th doctor done!
1200th doctor done!
1250th doctor done!
1300th doctor done!
1350th doctor done!
1400th doctor done!
1450th doctor done!
1500th doctor done!
1550th doctor done!
1600th doctor done!
1650th doctor done!
1700th doctor done!
1750th doctor done!
1800th doctor done!
1850th doctor done!
1900th doctor done!
1950th doctor done!
2000th doctor done!
2050th doctor done!
2100th doctor done!
2150th doctor done!
2200th doctor done!
2250th doctor done!
2300th doctor done!
2350th doctor done!
2400th doctor done!
2450th doctor done!
2500th doctor done!
2550th doctor done!


In [13]:
for url in all_doctors:
    scrape_doctor_info(url)

https://www.vaidam.com/doctors/dr-ali-enver-kurt


UnboundLocalError: cannot access local variable 'i' where it is not associated with a value

In [6]:
len(results)


14203

In [7]:
results[0]

{'name': 'Dr. Hayat İzel  Chat',
 'location': 'Lefkosa, Cyprus',
 'specialty': 'Gynaecologist and Obstetrician  IVF Specialist',
 'experience': '18 years of experience',
 'description': 'Dr. Hayat İzel is among the renowned Obstetricians and Gynecologists in Cyprus. He has 18\xa0years of experience in his field. He\xa0graduated from Cerrahpaşa Medical Faculty in 1988 and\xa0completed his gynecology and obstetrics specialization at Süleymaniye Gynecology and Obstetrics Training and Research Hospital in 1993. He is an active member of the\xa0Reproductive Health and Infertility Association,\xa0Reproductive Medicine and Surgery Association, and\xa0the Turkish Society of Gynecology and Obstetrics. He aims to provide the best medical treatments to every patient.'}

In [5]:
# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(results)

# Save the DataFrame to an Excel file
output_file = "data.xlsx"
df.to_excel(output_file, index=False)